In [2]:
import os
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sys
import scipy

In [3]:
DF=pd.read_csv('toefl.csv')
DF

,id,gender,listen,grammar,reading
0,1,FEMALE,40,50,56
1,2,FEMALE,33,44,46
2,3,MALE,39,44,47
3,4,FEMALE,44,55,66
4,5,MALE,43,56,53
...,...,...,...,...,...
95,96,FEMALE,43,48,56
96,97,MALE,50,54,60
97,98,MALE,55,50,67
98,99,FEMALE,53,61,66


In [4]:
DF['listen'].groupby(DF['gender']).describe()


,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
FEMALE,59.0,45.949153,6.929258,33.0,42.0,45.0,50.5,64.0
MALE,41.0,46.317073,6.954995,29.0,42.0,46.0,52.0,60.0


In [5]:
DF['grammar'].groupby(DF['gender']).describe()


,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
FEMALE,59.0,49.711864,7.808154,33.0,44.0,49.0,54.5,65.0
MALE,41.0,52.341463,7.831378,35.0,47.0,52.0,58.0,67.0


In [6]:

DF['reading'].groupby(DF['gender']).describe()

,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
FEMALE,59.0,54.677966,7.598640,38.0,51.0,55.0,60.0,75.0
MALE,41.0,54.341463,7.213216,41.0,48.0,53.0,60.0,67.0


In [25]:
DF['grade']=np.round(((DF['listen']+DF['grammar']+DF['reading'])/3)*10,1)
DF['grade'].groupby(DF['gender']).describe()

,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
FEMALE,59.0,501.130508,64.027468,350.0,460.0,490.0,538.35,640.0
MALE,41.0,509.997561,62.578712,350.0,466.7,506.7,550.00,646.7


In [26]:
DF['grade'].describe()

count    100.000000
mean     504.766000
std       63.270919
min      350.000000
25%      462.475000
50%      498.350000
75%      544.150000
max      646.700000
Name: grade, dtype: float64

In [8]:
DF['grade'].mean()

504.76600000000013

In [9]:
stats.ttest_1samp(DF['grade'],518)

Ttest_1sampResult(statistic=-2.091640251369215, pvalue=0.03903005193485989)

In [10]:
stats.ttest_1samp(DF.grade[DF.gender=='MALE'],530)

Ttest_1sampResult(statistic=-2.0466720589363137, pvalue=0.047300476144658675)

In [11]:
stats.ttest_1samp(DF.grade[DF.gender=='FEMALE'],524)

Ttest_1sampResult(statistic=-2.7435708998459476, pvalue=0.008073254314829163)

In [12]:
mu=518
n=100
xmean=DF['grade'].mean()
sigma=DF['grade'].std()
sem=sigma/np.sqrt(n)
zstat=(xmean-mu)/sem
pval=scipy.stats.norm.cdf(abs(zstat))

alpha=0.05
zq=np.abs(scipy.stats.norm.ppf(alpha/2))
me=zq*sem
lb=xmean-me
ub=xmean+me

np.round([n,xmean,sigma,sem,zstat,pval],4)

array([100.    , 504.766 ,  63.2709,   6.3271,  -2.0916,   0.9818])

In [13]:
def onesamplez(n,mean,sigma,mu=0,alt='two',conf_level=0.95):
    import scipy
    sem= sigma/np.sqrt(n)
    zstat=(mean-mu)/sem
    alpha=1-conf_level
    
    if alt=='less':
        pval=scipy.stats.norm.cdf(zstat)
        H1='H1:mu<'+str(mu)
        
    elif alt=='greater':
        pval=1-scipy.stats.norm.cdf(zstat)
        H1='H1:mu<'+str(mu)
        
    else:
        pval=2*scipy.stats.norm.cdf(zstat)
        H1='H1:mu<'+str(mu)
        
    me= scipy.stats.norm.ppf(1-alpha/2)*sem
    lb=xmean-me
    ub=xmean+me
    
    return pd.DataFrame(
    [[n, mean, sigma, sem, zstat, pval, me, lb, ub, alpha]],
    columns=['n', 'mean', 'sigma', 'sem', 'zstat', 'pval', 'me', 'lb', 'ub', 'alpha'],
        index=[H1])

        

In [14]:
stats.binom_test(x=79,n=500,p=0.2,alternative='less')

0.009506417149632919

In [17]:
onesamplez(6,17.416667,1.679782,19,alt='less',conf_level=0.95)

,n,mean,sigma,sem,zstat,pval,me,lb,ub,alpha
H1:mu<19,6,17.416667,1.679782,0.685768,-2.308846,0.010476,1.344081,503.421919,506.110081,0.05
